In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_datasets as tfds

os.environ['TFDS_DIR'] = '/longterm/kleino/tfds/'

from tensorflow.data import Dataset
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Activation
from tensorflow.keras.models import Model

import gloro.v1 as gloro

from gloro.v1 import GloroNet
from gloro.v1.layers import MinMax

from dos_attacks import GloroDosAttack, RsDosUnderlying, RsDosAttack

from gloro.smoothing.models import SmoothedModel

sys.path.append('../nf')
sys.path.append('../nf/training')

from training.utils import get_data

tf.keras.backend.set_image_data_format('channels_last')

# Select the GPU and allow memory growth to avoid taking all the RAM.
gpu = 2

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[gpu], 'GPU')

for device in tf.config.experimental.get_visible_devices('GPU'):
  tf.config.experimental.set_memory_growth(device, True)

# GloRo

## CIFAR-10 $\epsilon = 0.141$

In [2]:
attack = GloroDosAttack(0.141)

In [3]:
train, test, meta = get_data('cifar10', batch_size=256)

In [4]:
g = GloroNet.load_model(
  '/longterm/kleino/repos/lib-gloro/experiments/nf/training/models/gloro.528cc012921237e6.gloronet')

In [5]:
vra_under_dos, fraction_susceptible = attack.eval(g, test)

print(
  f'vra_under_dos: {vra_under_dos.numpy():.3f}\n'
  f'fraction_susceptible: {fraction_susceptible.numpy():.3f}')

attacking at a radius of 2 * epsilon...
80/80 [==============================] - ETA: 0s - vra: 0.6019 - dos_success: 0.4169 ETA:  - 4s 52ms/step - vra: 0.6019 - dos_success: 0.4189
vra_under_dos: 0.534
fraction_susceptible: 0.114


## MNIST $\epsilon = 1.58$

In [3]:
attack = GloroDosAttack(1.58)

train, test, meta = get_data('mnist', batch_size=256)

g = GloroNet.load_model(
  '/longterm/kleino/repos/lib-gloro/experiments/nf/training/models/gloro.077647b1c09ced0d.gloronet')

vra_under_dos, fraction_susceptible = attack.eval(g, test)

print(
  f'vra_under_dos: {vra_under_dos.numpy():.3f}\n'
  f'fraction_susceptible: {fraction_susceptible.numpy():.3f}')

attacking at a radius of 2 * epsilon...
80/80 [==============================] - 21s 267ms/step - vra: 0.6060 - dos_success: 0.7361
vra_under_dos: 0.266
fraction_susceptible: 0.561


# Randomized Smoothing

## CIFAR-10 $\epsilon = 1.58$

In [2]:
f = tf.keras.models.load_model(
  '/longterm/kleino/repos/lib-gloro/experiments/rtk/models/smoothed.architecture-cnn_6C2F.batch_size-256'
  '.dataset-cifar10.epochs-50.lr_schedule-decay_to_0.00001_after_half.sigma-0.125.h5')

In [3]:
g = SmoothedModel(model=f, sigma=0.125, n=100000)

In [4]:
samples = 100
batch_size = 10

train, test, meta = get_data('cifar10', batch_size=batch_size)

iterator = iter(tfds.as_numpy(test))
new_test_x, new_test_y = [], []
for _ in range(samples // batch_size):
  x, y = next(iterator)
  new_test_x.append(x)
  new_test_y.append(y)
  
test = Dataset.from_tensor_slices(
  (tf.concat(new_test_x, axis=0), tf.concat(new_test_y, axis=0))).cache().batch(batch_size)

#### Using underlying model

In [6]:
attack = RsDosUnderlying(0.141)

vra_under_dos, fraction_susceptible = attack.eval(g, test)

print(
  f'vra_under_dos: {vra_under_dos.numpy():.3f}\n'
  f'fraction_susceptible: {fraction_susceptible.numpy():.3f}')

attacking underlying model at a radius of 2 * epsilon...
20/20 [==============================] - 500s 25s/step - vra: 0.6000 - dos_success: 0.3300
vra_under_dos: 0.560
fraction_susceptible: 0.067


#### Using smoothed PGD

In [5]:
attack = RsDosAttack(0.141, aggregate_before_grad=True)

vra_under_dos, fraction_susceptible = attack.eval(g, test)

print(
  f'vra_under_dos: {vra_under_dos.numpy():.3f}\n'
  f'fraction_susceptible: {fraction_susceptible.numpy():.3f}')

attacking underlying model at a radius of 2 * epsilon...
20/20 [==============================] - 503s 25s/step - vra: 0.5900 - dos_success: 0.2700
vra_under_dos: 0.510
fraction_susceptible: 0.136


In [10]:
attack = RsDosAttack(0.141, samples=1000, aggregate_before_grad=True)

vra_under_dos, fraction_susceptible = attack.eval(g, test)

print(
  f'vra_under_dos: {vra_under_dos.numpy():.3f}\n'
  f'fraction_susceptible: {fraction_susceptible.numpy():.3f}')

attacking underlying model at a radius of 2 * epsilon...
20/20 [==============================] - 487s 24s/step - vra: 0.5900 - dos_success: 0.2700
vra_under_dos: 0.510
fraction_susceptible: 0.136


#### Using boundary loss and smoothed PGD

In [7]:
def boundary_loss(y_true, y_pred):
  values, indices = tf.math.top_k(y_pred, 2)

  return -(values[:,0] - values[:,1])**2

In [8]:
attack = RsDosAttack(0.141, aggregate_before_grad=True, project_back=False, loss=boundary_loss)

vra_under_dos, fraction_susceptible = attack.eval(g, test)

print(
  f'vra_under_dos: {vra_under_dos.numpy():.3f}\n'
  f'fraction_susceptible: {fraction_susceptible.numpy():.3f}')

attacking underlying model at a radius of 2 * epsilon...
20/20 [==============================] - 497s 25s/step - vra: 0.5900 - dos_success: 0.4300
vra_under_dos: 0.510
fraction_susceptible: 0.136


#### CIFAR-10 $\epsilon = 0.25$

In [9]:
attack = RsDosAttack(0.25, aggregate_before_grad=True)

vra_under_dos, fraction_susceptible = attack.eval(g, test)

print(
  f'vra_under_dos: {vra_under_dos.numpy():.3f}\n'
  f'fraction_susceptible: {fraction_susceptible.numpy():.3f}')

attacking underlying model at a radius of 2 * epsilon...
20/20 [==============================] - 495s 25s/step - vra: 0.5100 - dos_success: 0.4400
vra_under_dos: 0.380
fraction_susceptible: 0.255
